In [0]:
spark.sql("DROP TABLE IF EXISTS fatos_clubes")
spark.sql("DROP TABLE IF EXISTS dim_jogadores")
spark.sql("DROP TABLE IF EXISTS dim_clubes")
spark.sql("DROP TABLE IF EXISTS dim_partidas")

Out[11]: DataFrame[]

Importação de Funções e Carregamento das Tabelas Silver

In [0]:
from pyspark.sql import functions as F

# Carregar tabelas Silver
cartoes_silver_df = spark.table("campeonato_brasileiro_cartoes_silver")
gols_silver_df = spark.table("campeonato_brasileiro_gols_silver")
estatisticas_silver_df = spark.table("campeonato_brasileiro_estatisticas_silver")
full_silver_df = spark.table("campeonato_brasileiro_full_silver")

Criação da Tabela de Fatos fatos_clubes:

In [0]:
# Tabela de Fatos: fatos_clubes
fatos_clubes_df = cartoes_silver_df.groupBy("clube").agg(
    F.count("partida_id").alias("total_partidas_cartoes"),
    F.sum(F.when(F.col("cartao") == 'Amarelo', 1).otherwise(0)).alias("total_cartoes_amarelos"),
    F.sum(F.when(F.col("cartao") == 'Vermelho', 1).otherwise(0)).alias("total_cartoes_vermelhos")
)

fatos_vitorias_df = full_silver_df.groupBy("vencedor").agg(
    F.count("ID").alias("total_vitorias")
).withColumnRenamed("vencedor", "clube")

# Combinar todas as tabelas de fatos em uma única tabela de fatos
fatos_clubes_completo_df = fatos_clubes_df.join(fatos_vitorias_df, "clube", "left")

# Calcular as médias necessárias
fatos_clubes_completo_df = fatos_clubes_completo_df.withColumn(
    "media_cartoes_amarelos", F.col("total_cartoes_amarelos") / F.col("total_partidas_cartoes")
).withColumn(
    "media_cartoes_vermelhos", F.col("total_cartoes_vermelhos") / F.col("total_partidas_cartoes")
)

Criação das Tabela de dimensão




In [0]:
# Criar tabelas de dimensão
dim_jogadores_df = cartoes_silver_df.select("atleta", "clube", "num_camisa", "posicao").distinct()
dim_clubes_df = full_silver_df.select(F.col("mandante").alias("clube"), "arena").distinct()
dim_partidas_df = full_silver_df.select("ID", "data", "hora", "mandante", "visitante", "mandante_Placar", "visitante_Placar").distinct()

# Verificar valores nulos nas tabelas de dimensão
exibir_valores_nulos(fatos_clubes_completo_df)
exibir_valores_nulos(dim_jogadores_df)
exibir_valores_nulos(dim_clubes_df)
exibir_valores_nulos(dim_partidas_df)


+-----+----------------------+----------------------+-----------------------+--------------+----------------------+-----------------------+
|clube|total_partidas_cartoes|total_cartoes_amarelos|total_cartoes_vermelhos|total_vitorias|media_cartoes_amarelos|media_cartoes_vermelhos|
+-----+----------------------+----------------------+-----------------------+--------------+----------------------+-----------------------+
|    0|                     0|                     0|                      0|             0|                     0|                      0|
+-----+----------------------+----------------------+-----------------------+--------------+----------------------+-----------------------+

+------+-----+----------+-------+
|atleta|clube|num_camisa|posicao|
+------+-----+----------+-------+
|     0|    0|         0|      0|
+------+-----+----------+-------+

+-----+-----+
|clube|arena|
+-----+-----+
|    0|    0|
+-----+-----+

+---+----+----+--------+---------+---------------+-------

In [0]:
# Criar tabelas temporárias Gold
fatos_clubes_completo_df.createOrReplaceTempView("fatos_clubes")
dim_jogadores_df.createOrReplaceTempView("dim_jogadores")
dim_clubes_df.createOrReplaceTempView("dim_clubes")
dim_partidas_df.createOrReplaceTempView("dim_partidas")

# Salvar os DataFrames como tabelas permanentes em formato Parquet
fatos_clubes_completo_df.write.format("parquet").mode("overwrite").saveAsTable("fatos_clubes")
dim_jogadores_df.write.format("parquet").mode("overwrite").saveAsTable("dim_jogadores")
dim_clubes_df.write.format("parquet").mode("overwrite").saveAsTable("dim_clubes")
dim_partidas_df.write.format("parquet").mode("overwrite").saveAsTable("dim_partidas")

print("Tabelas Gold criadas com sucesso.")

Tabelas Gold criadas com sucesso.
